## التضمينات

في المثال السابق، عملنا على متجهات حقيبة الكلمات عالية الأبعاد بطول `vocab_size`، وقمنا بتحويل متجهات التمثيل الموضعي منخفضة الأبعاد بشكل صريح إلى تمثيل نادر باستخدام الترميز الواحد. هذا التمثيل الواحد ليس فعالاً من حيث الذاكرة. بالإضافة إلى ذلك، يتم التعامل مع كل كلمة بشكل مستقل عن الأخرى، لذا فإن المتجهات المشفرة باستخدام الترميز الواحد لا تعبر عن التشابهات الدلالية بين الكلمات.

في هذه الوحدة، سنواصل استكشاف مجموعة بيانات **News AG**. للبدء، دعونا نقوم بتحميل البيانات ونسترجع بعض التعريفات من الوحدة السابقة.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### ما هو التضمين؟

فكرة **التضمين** هي تمثيل الكلمات باستخدام متجهات كثيفة ذات أبعاد أقل تعكس المعنى الدلالي للكلمة. سنتحدث لاحقًا عن كيفية بناء تضمينات كلمات ذات معنى، ولكن في الوقت الحالي دعونا نفكر في التضمينات كوسيلة لتقليل أبعاد متجه الكلمة.

لذلك، تأخذ طبقة التضمين كلمة كمدخل، وتنتج متجهًا كخرج بحجم `embedding_size` المحدد. من ناحية ما، تشبه طبقة `Dense`، ولكن بدلاً من أخذ متجه مشفر بطريقة one-hot كمدخل، يمكنها أخذ رقم الكلمة.

باستخدام طبقة التضمين كأول طبقة في شبكتنا، يمكننا الانتقال من نموذج حقيبة الكلمات إلى نموذج **حقيبة التضمينات**، حيث نقوم أولاً بتحويل كل كلمة في النص إلى التضمين المقابل لها، ثم نحسب دالة تجميع معينة على جميع تلك التضمينات، مثل `sum` أو `average` أو `max`.

![صورة توضح مصنف تضمين لخمس كلمات متتالية.](../../../../../translated_images/ar/embedding-classifier-example.b77f021a7ee67eee.webp)

تتكون شبكة التصنيف العصبية الخاصة بنا من الطبقات التالية:

* طبقة `TextVectorization`، التي تأخذ سلسلة نصية كمدخل، وتنتج موترًا يحتوي على أرقام الرموز. سنحدد حجم مفردات معقول `vocab_size`، ونتجاهل الكلمات الأقل استخدامًا. سيكون شكل المدخل 1، وشكل الخرج $n$، حيث سنحصل على $n$ رمزًا كنتيجة، كل منها يحتوي على أرقام من 0 إلى `vocab_size`.
* طبقة `Embedding`، التي تأخذ $n$ أرقام، وتقلل كل رقم إلى متجه كثيف بطول معين (100 في مثالنا). وبالتالي، سيتم تحويل موتر المدخل الذي شكله $n$ إلى موتر شكله $n\times 100$.
* طبقة التجميع، التي تأخذ متوسط هذا الموتر على طول المحور الأول، أي أنها ستحسب متوسط جميع موترات المدخل $n$ التي تمثل كلمات مختلفة. لتنفيذ هذه الطبقة، سنستخدم طبقة `Lambda`، ونمرر إليها الدالة لحساب المتوسط. سيكون شكل الخرج 100، وسيكون التمثيل الرقمي للتسلسل النصي بالكامل.
* المصنف الخطي النهائي `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


في ملخص الطباعة، في عمود **شكل الإخراج**، يشير البُعد الأول للتنسور `None` إلى حجم الدفعة الصغيرة، بينما يشير البُعد الثاني إلى طول تسلسل الرموز. جميع تسلسلات الرموز في الدفعة الصغيرة لها أطوال مختلفة. سنتحدث عن كيفية التعامل مع ذلك في القسم التالي.

الآن دعونا ندرب الشبكة:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **ملاحظة** أننا نقوم ببناء المتجهات بناءً على مجموعة فرعية من البيانات. يتم ذلك لتسريع العملية، وقد يؤدي ذلك إلى حالة حيث لا تكون جميع الرموز من نصنا موجودة في المفردات. في هذه الحالة، سيتم تجاهل تلك الرموز، مما قد يؤدي إلى دقة أقل قليلاً. ومع ذلك، في الحياة الواقعية، غالبًا ما تعطي مجموعة فرعية من النص تقديرًا جيدًا للمفردات.


### التعامل مع أحجام تسلسل المتغيرات

لنفهم كيف تتم عملية التدريب في مجموعات صغيرة. في المثال أعلاه، يكون للمدخلات شكل بعد واحد، ونستخدم مجموعات صغيرة بطول 128، بحيث يكون الحجم الفعلي للمدخلات هو $128 \times 1$. ومع ذلك، يختلف عدد الرموز في كل جملة. إذا قمنا بتطبيق طبقة `TextVectorization` على مدخل واحد، فإن عدد الرموز الناتجة يختلف بناءً على كيفية تقسيم النص إلى رموز:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


ومع ذلك، عندما نطبق المُوجِّه على عدة تسلسلات، يجب أن يُنتِج موترًا ذو شكل مستطيل، لذلك يملأ العناصر غير المستخدمة برمز PAD (والذي في حالتنا هو صفر):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

هنا يمكننا رؤية التضمينات:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **ملاحظة**: لتقليل كمية الحشو، في بعض الحالات يكون من المنطقي ترتيب جميع التسلسلات في مجموعة البيانات حسب زيادة الطول (أو بشكل أدق، حسب عدد الرموز). سيضمن ذلك أن يحتوي كل دفعة صغيرة على تسلسلات ذات أطوال متشابهة.


## التضمينات الدلالية: Word2Vec

في المثال السابق، تعلمت طبقة التضمين كيفية تحويل الكلمات إلى تمثيلات متجهة، ولكن هذه التمثيلات لم تكن تحمل معنى دلاليًا. سيكون من الجيد تعلم تمثيل متجه بحيث تكون الكلمات المتشابهة أو المرادفات قريبة من بعضها البعض من حيث مسافة المتجه (على سبيل المثال، المسافة الإقليدية).

للقيام بذلك، نحتاج إلى تدريب نموذج التضمين مسبقًا على مجموعة كبيرة من النصوص باستخدام تقنية مثل [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). تعتمد هذه التقنية على بنيتين رئيسيتين تُستخدمان لإنتاج تمثيل موزع للكلمات:

- **الحقيبة المستمرة للكلمات** (CBoW)، حيث نقوم بتدريب النموذج للتنبؤ بكلمة من السياق المحيط. بالنظر إلى النغرام $(W_{-2},W_{-1},W_0,W_1,W_2)$، هدف النموذج هو التنبؤ بـ $W_0$ من $(W_{-2},W_{-1},W_1,W_2)$.
- **التخطي المستمر للكلمات** هو عكس CBoW. يستخدم النموذج نافذة الكلمات المحيطة بالسياق للتنبؤ بالكلمة الحالية.

CBoW أسرع، بينما التخطي المستمر أبطأ ولكنه يقوم بتمثيل الكلمات النادرة بشكل أفضل.

![صورة توضح كلا من خوارزميات CBoW و Skip-Gram لتحويل الكلمات إلى متجهات.](../../../../../translated_images/ar/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

لتجربة تضمين Word2Vec المدرب مسبقًا على مجموعة بيانات أخبار Google، يمكننا استخدام مكتبة **gensim**. أدناه نجد الكلمات الأكثر تشابهًا مع كلمة 'neural'.

> **ملاحظة:** عند إنشاء متجهات الكلمات لأول مرة، قد يستغرق تنزيلها بعض الوقت!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


يمكننا أيضًا استخراج تضمين المتجه من الكلمة، لاستخدامه في تدريب نموذج التصنيف. يحتوي التضمين على 300 مكون، ولكن هنا نعرض فقط أول 20 مكونًا من المتجه للتوضيح:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

الشيء الرائع بشأن التضمينات الدلالية هو أنه يمكنك التلاعب بترميز المتجه بناءً على الدلالات. على سبيل المثال، يمكننا أن نطلب العثور على كلمة يكون تمثيلها المتجهي قريبًا قدر الإمكان من الكلمات *ملك* و *امرأة*، وبعيدًا قدر الإمكان عن كلمة *رجل*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

يستخدم المثال أعلاه بعض السحر الداخلي لـ GenSym، لكن المنطق الأساسي بسيط جدًا في الواقع. الشيء المثير للاهتمام حول التضمينات هو أنه يمكنك إجراء عمليات المتجه العادية على متجهات التضمين، وسيعكس ذلك العمليات على **معاني** الكلمات. يمكن التعبير عن المثال أعلاه من حيث عمليات المتجه: نحسب المتجه المقابل لـ **ملك-رجل+امرأة** (تُجرى العمليات `+` و `-` على تمثيلات المتجهات للكلمات المقابلة)، ثم نجد الكلمة الأقرب في القاموس لذلك المتجه:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **ملاحظة**: اضطررنا إلى إضافة معاملات صغيرة إلى متجهات *man* و *woman* - حاول إزالة هذه المعاملات لترى ما سيحدث.

للعثور على أقرب متجه، نستخدم أدوات TensorFlow لحساب متجه المسافات بين متجهنا وجميع المتجهات في المفردات، ثم نحدد مؤشر الكلمة الأدنى باستخدام `argmin`.


بينما يبدو أن Word2Vec طريقة رائعة للتعبير عن دلالات الكلمات، إلا أنه يعاني من العديد من العيوب، بما في ذلك ما يلي:

* كلا النموذجين CBoW و skip-gram هما **تضمينات تنبؤية**، ويأخذان في الاعتبار السياق المحلي فقط. Word2Vec لا يستفيد من السياق العام.
* Word2Vec لا يأخذ في الاعتبار **مورفولوجيا** الكلمة، أي حقيقة أن معنى الكلمة يمكن أن يعتمد على أجزاء مختلفة منها، مثل الجذر.

**FastText** يحاول التغلب على القيد الثاني، ويبني على Word2Vec من خلال تعلم تمثيلات المتجهات لكل كلمة والن-غرامات الحرفية الموجودة داخل كل كلمة. يتم بعد ذلك حساب متوسط قيم التمثيلات في متجه واحد في كل خطوة تدريب. على الرغم من أن هذا يضيف الكثير من العمليات الحسابية الإضافية أثناء التدريب المسبق، إلا أنه يمكّن تضمينات الكلمات من تشفير معلومات الأجزاء الفرعية للكلمة.

طريقة أخرى، **GloVe**، تستخدم نهجًا مختلفًا لتضمينات الكلمات، يعتمد على تحليل مصفوفة سياق الكلمات. أولاً، يقوم ببناء مصفوفة كبيرة تحصي عدد مرات ظهور الكلمات في سياقات مختلفة، ثم يحاول تمثيل هذه المصفوفة في أبعاد أقل بطريقة تقلل من خسارة إعادة البناء.

مكتبة gensim تدعم هذه التضمينات، ويمكنك تجربتها عن طريق تغيير كود تحميل النموذج أعلاه.


## استخدام التضمينات المدربة مسبقًا في Keras

يمكننا تعديل المثال أعلاه لملء المصفوفة في طبقة التضمين الخاصة بنا بتضمينات دلالية، مثل Word2Vec. من المحتمل أن لا تتطابق مفردات التضمين المدرب مسبقًا مع مفردات النص، لذا علينا اختيار واحدة. هنا نستكشف الخيارين الممكنين: استخدام مفردات الـ tokenizer، واستخدام المفردات من تضمينات Word2Vec.

### استخدام مفردات الـ tokenizer

عند استخدام مفردات الـ tokenizer، بعض الكلمات من المفردات سيكون لها تضمينات Word2Vec المقابلة، وبعضها سيكون مفقودًا. بالنظر إلى أن حجم المفردات لدينا هو `vocab_size`، وطول متجه تضمين Word2Vec هو `embed_size`، فإن طبقة التضمين سيتم تمثيلها بمصفوفة أوزان ذات شكل `vocab_size`$\times$`embed_size`. سنقوم بملء هذه المصفوفة من خلال المرور عبر المفردات:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


بالنسبة للكلمات التي لا توجد في مفردات Word2Vec، يمكننا إما تركها كأصفار، أو إنشاء متجه عشوائي.

الآن يمكننا تعريف طبقة التضمين باستخدام الأوزان المدربة مسبقًا:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **ملاحظة**: لاحظ أننا قمنا بتعيين `trainable=False` عند إنشاء `Embedding`، مما يعني أننا لن نقوم بإعادة تدريب طبقة الـ Embedding. قد يؤدي ذلك إلى انخفاض طفيف في الدقة، ولكنه يسرّع عملية التدريب.

### استخدام مفردات التضمين

إحدى المشكلات في النهج السابق هي أن المفردات المستخدمة في `TextVectorization` و `Embedding` مختلفة. للتغلب على هذه المشكلة، يمكننا استخدام أحد الحلول التالية:
* إعادة تدريب نموذج Word2Vec على مفرداتنا.
* تحميل بياناتنا باستخدام المفردات من نموذج Word2Vec المدرب مسبقًا. يمكن تحديد المفردات المستخدمة لتحميل البيانات أثناء عملية التحميل.

يبدو أن النهج الأخير أسهل، لذا دعونا نقوم بتنفيذه. أولاً، سنقوم بإنشاء طبقة `TextVectorization` مع المفردات المحددة، المأخوذة من تضمينات Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

تحتوي مكتبة تضمين الكلمات gensim على وظيفة مريحة، `get_keras_embeddings`، والتي ستقوم تلقائيًا بإنشاء طبقة تضمين Keras المقابلة لك.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


أحد الأسباب التي تجعلنا لا نرى دقة أعلى هو أن بعض الكلمات من مجموعة البيانات الخاصة بنا مفقودة في مفردات GloVe المدربة مسبقًا، وبالتالي يتم تجاهلها بشكل أساسي. للتغلب على ذلك، يمكننا تدريب التضمينات الخاصة بنا بناءً على مجموعة البيانات الخاصة بنا.


## التضمينات السياقية

أحد القيود الرئيسية لتمثيلات التضمينات المدربة مسبقًا مثل Word2Vec هو أنها، على الرغم من قدرتها على التقاط بعض معاني الكلمة، لا تستطيع التمييز بين المعاني المختلفة. يمكن أن يتسبب هذا في مشاكل في النماذج اللاحقة.

على سبيل المثال، كلمة "play" لها معانٍ مختلفة في الجملتين التاليتين:
- ذهبت إلى **عرض مسرحي** في المسرح.
- جون يريد أن **يلعب** مع أصدقائه.

التضمينات المدربة مسبقًا التي تحدثنا عنها تمثل كلا المعنيين لكلمة "play" بنفس التضمين. للتغلب على هذا القيد، نحتاج إلى بناء تضمينات تعتمد على **نموذج اللغة**، الذي يتم تدريبه على مجموعة كبيرة من النصوص، و*يعرف* كيف يمكن للكلمات أن تتجمع معًا في سياقات مختلفة. مناقشة التضمينات السياقية خارج نطاق هذا الدرس، لكننا سنعود إليها عند الحديث عن نماذج اللغة في الوحدة التالية.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
